Finding Python API information here: https://open-meteo.com/en/docs?latitude=19.7297&longitude=-155.09&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,showers,evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m

Directions: 
1) install openmeteo-requests
2) install requests-cache retry-requests

imports needed to access the API:
```python
import pandas as pd
import requests_cache
from retry_requests import retry
```

Example code template given from the API_docs page:
```python
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 19.7297,
	"longitude": -155.09,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "showers", "evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_80m", "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m"],
}
responses = openmeteo.weather_api(url, params=params)
```

In [10]:
# following the tutorial above, starting with appropriate imports:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [11]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 19.7297,
	"longitude": -155.09,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "showers", "evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_80m", "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m"],
}
responses = openmeteo.weather_api(url, params=params)

Everything seems to work! Not familiar with this API yet, doing some exploring:

In [12]:
# what format is responses in?
type(responses)

list

In [13]:
# check the length of the list and look at the first item
print(len(responses))
responses[0]

1


Ah, ok! So it seems the template builds it this way so that one could put in different cities and get a list of different responses. In this case, there is only one. So we can just convert that single response to json for our data.

In [14]:
# set data to our response for Hilo, Hawaii
data = responses[0]

# now I can access any of the parameters we called for - in this example hourly data - the first variable is relative humidity 2m
hourly = data.Hourly()
hourly.Variables(1).ValuesAsNumpy()

array([ 66.,  73.,  76.,  77.,  83.,  89.,  94.,  96.,  96.,  90.,  82.,
        78.,  77.,  78.,  79.,  80.,  80.,  77.,  69.,  70.,  67.,  63.,
        61.,  57.,  54.,  57.,  62.,  69.,  75.,  81.,  83.,  83.,  81.,
        75.,  70.,  69.,  70.,  72.,  72.,  72.,  73.,  73.,  71.,  75.,
        78.,  82.,  78.,  76.,  76.,  77.,  81.,  82.,  85.,  90.,  94.,
        95.,  97., 100., 100.,  99.,  96.,  96.,  96.,  95.,  93.,  91.,
        89.,  83.,  77.,  73.,  66.,  64.,  67.,  72.,  77.,  82.,  88.,
        91.,  94.,  97.,  99., 100., 100., 100.,  98.,  92.,  84.,  78.,
        77.,  77.,  77.,  74.,  71.,  67.,  63.,  59.,  57.,  60.,  65.,
        71.,  79.,  88.,  94.,  93.,  88.,  84.,  82.,  80.,  78.,  76.,
        75.,  73.,  70.,  67.,  64.,  60.,  56.,  53.,  52.,  52.,  55.,
        61.,  70.,  78.,  84.,  89.,  92.,  90.,  85.,  82.,  82.,  85.,
        86.,  86.,  86.,  85.,  83.,  80.,  76.,  70.,  63.,  59.,  58.,
        60.,  63.,  68.,  74.,  80.,  86.,  91.,  9

In [15]:
# and to be able to link with other data based on location and time, I need to also retrieve the lat/long information
lat = data.Latitude()
long = data.Longitude()
elevation = data.Elevation()


All in all this API is very clear and easy to use, with extremely helpful documentation. The code below was generated through the OpenMeteo API docs website after giving it the parameters I wanted to check for. 

In [16]:
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_showers = hourly.Variables(4).ValuesAsNumpy()
hourly_evapotranspiration = hourly.Variables(5).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_80m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(10).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(11).ValuesAsNumpy()
hourly_wind_direction_80m = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_direction_120m = hourly.Variables(13).ValuesAsNumpy()
hourly_wind_direction_180m = hourly.Variables(14).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(15).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["evapotranspiration"] = hourly_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_80m"] = hourly_wind_direction_80m
hourly_data["wind_direction_120m"] = hourly_wind_direction_120m
hourly_data["wind_direction_180m"] = hourly_wind_direction_180m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.head()

,date,temperature_2m,relative_humidity_2m,precipitation,rain,showers,evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m
0,2025-09-25 00:00:00+00:00,23.906500,66.0,0.0,0.0,0.0,0.20,1.008685,12.889810,16.992609,17.208603,16.923830,54.090195,53.615551,52.650585,51.911175,32.760002
1,2025-09-25 01:00:00+00:00,22.856501,73.0,0.0,0.0,0.0,0.13,0.751881,11.885453,15.696165,15.913465,16.135872,54.865723,53.392834,52.352322,51.340172,30.960001
2,2025-09-25 02:00:00+00:00,22.306499,76.0,0.0,0.0,0.0,0.09,0.646420,10.233123,13.513193,14.003028,14.003028,50.710617,48.239799,46.041542,43.958473,27.719999
3,2025-09-25 03:00:00+00:00,21.856501,77.0,0.0,0.0,0.0,0.06,0.602765,8.669949,12.218805,12.475961,12.496718,48.366554,44.999897,43.830952,41.496380,24.119999
4,2025-09-25 04:00:00+00:00,20.956501,83.0,0.0,0.0,0.0,0.03,0.421652,5.860375,9.178235,9.220499,9.290511,47.489597,41.820091,38.659828,35.537766,20.519999


In [17]:
hourly_dataframe.to_csv("../../Data/Raw/openmeteo_sample_9_24_25.csv")